In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns
import sklearn



In [ ]:
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.cluster import hierarchy

## Questions.csv add whether bundle or not var and whether listening 0 or reading 1 ##

In [2]:
rawdata = pd.read_csv('train.csv')

In [3]:
ques=pd.read_csv('questions.csv')
lec=pd.read_csv('lectures.csv')

In [5]:
ques.head()

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


In [92]:
ques.shape[0]
a = ques['part']>4

In [93]:
partc=[0 for x in range(ques.shape[0])]
c=partc+a
c
ques['partc']=c

In [196]:
ques['bundlec']=0
d=ques[ques.duplicated(['bundle_id'])]

In [170]:
b=[0  for x in range(ques.shape[0])]

for x in range(len(ques['bundle_id'])):
    b[x]= ques['bundle_id'][x] in d[['bundle_id']].values
len(b)

13523

In [171]:
ques['bundlec']=b
ques['bundlec']=ques['bundlec'].astype(int)
ques.head()

,question_id,bundle_id,correct_answer,part,tags,partc,bundlec
0,0,0,0,1,51 131 162 38,0,0
1,1,1,1,1,131 36 81,0,0
2,2,2,0,1,131 101 162 92,0,0
3,3,3,0,1,131 149 162 29,0,0
4,4,4,3,1,131 5 162 38,0,0


In [172]:
ques['bundlec'].sum()

5372

## Cluster Tags ##

In [199]:
import networkx as nx
import networkx.algorithms.community as nxcom


In [204]:
ques['tags'].isnull().sum()

1

In [209]:
ques['tags'] = ques['tags'].fillna(0)


In [214]:
ques['tags'] = ques['tags'].astype(str)
ques['tags'] = ques['tags'].apply(lambda x: np.array(x.split()).astype(int))

In [215]:
ques.head()

,question_id,bundle_id,correct_answer,part,tags,partc,bundlec
0,0,0,0,1,"[51, 131, 162, 38]",0,0
1,1,1,1,1,"[131, 36, 81]",0,0
2,2,2,0,1,"[131, 101, 162, 92]",0,0
3,3,3,0,1,"[131, 149, 162, 29]",0,0
4,4,4,3,1,"[131, 5, 162, 38]",0,0


In [221]:
tags=np.unique(np.concatenate(ques['tags'].values))
tags

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

## Lectures.csv add type numeric var and whether listening 0 or reading 1 ##

In [198]:
lec.head()

,lecture_id,tag,part,type_of,partc,type
0,89,159,5,concept,1,0
1,100,70,1,concept,0,0
2,185,45,6,concept,1,0
3,192,79,5,solving question,1,2
4,317,156,5,solving question,1,2


In [191]:
lec.shape[0]
aa = lec['part']>4
partcc=[0 for x in range(lec.shape[0])]
cc=partcc+aa
cc
lec['partc']=cc

In [192]:
lec.head()

,lecture_id,tag,part,type_of,partc
0,89,159,5,concept,1
1,100,70,1,concept,0
2,185,45,6,concept,1
3,192,79,5,solving question,1
4,317,156,5,solving question,1


In [195]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

y = label_encoder.fit_transform(lec['type_of'])
lec['type']=y
lec.head()

,lecture_id,tag,part,type_of,partc,type
0,89,159,5,concept,1,0
1,100,70,1,concept,0,0
2,185,45,6,concept,1,0
3,192,79,5,solving question,1,2
4,317,156,5,solving question,1,2


In [9]:
range(1,len(c))

range(1, 10)

In [10]:
c[10:]

[]

In [11]:
c[]

SyntaxError: invalid syntax (<ipython-input-11-2178f35de4f1>, line 1)

In [13]:
max(c[9:])

8

In [15]:
for i in range(1,10):
    print(i)

1
2
3
4
5
6
7
8
9


In [16]:
c[:1]

[1]

In [17]:
c[1:]

[3, 4, 2, 5, 6, 6, 7, 7, 8]

In [18]:
c[:2]

[1, 3]

In [27]:
c.count(5)

1

In [29]:
max(c[:1])

1

In [40]:
height=[0,1,0,2,1,0,1,3,2,1,2,1]

In [38]:
    l[1]=max(height[:2])

TypeError: 'int' object does not support item assignment

In [39]:
water=0
l=0
r=0
for i in range(1,10):
    l=max(height[:i+1])
    r=max(height[i:])
    water = water + min(l,r)-height[i]
print(water)

6


In [42]:
        water=0
        l=0
        r=len(height)-1
        max_r=max_l=0
        while l<r:
            if height[l]<height[r]:
                if height[l]>max_l:
                    max_l=height[l]
                else:
                    water = water+max_l-height[l]
                l = l+1
            else:
                if height[r]>max_r:
                    max_r=height[r]
                else:
                    water = water+max_r-height[r]
                r=r-1
        print(water)
                

6


In [43]:
c[:3]

[1, 3, 4]

In [44]:
c[1:3]

[3, 4]

In [45]:
c[1:]

[3, 4, 2, 5, 6, 6, 7, 7, 8]